<a href="https://colab.research.google.com/github/Remdeht/ia_detector/blob/master/IA_CDC_LDA_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Libraries, install github repo, and initiate GEE and Google Drive


In [ ]:
import ee

try:
  ee.Initialize()
except:
  ee.Authenticate()  # if first time user of the Google Earth Engine on this device, otherwise ee.Initialize() is enough

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Clone the Github Repo containing the GEE/Python code needed to run the classification of irrigated areas

In [ ]:
!git clone https://github.com/Remdeht/ia_detector.git -b threshold_testing ia
!pip install monthdelta
!pip install folium==0.10.1

In [ ]:
# Load libraries
import itertools, sys, os, folium
from IPython.display import display
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime 
from skimage.filters import threshold_otsu
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Load functions from the repo
from ia.gee_functions.classification_lda import create_feature_data, take_strat_sample, perform_lda_scaling, min_distance_classification, random_forest, remove_outliers
from ia.gee_functions.constants import GEE_USER_PATH
from ia.gee_functions.classification import classify_irrigated_areas, join_seasonal_irrigated_areas
from ia.gee_functions.export import track_task, export_to_drive, export_to_asset
from ia.gee_functions import visualization
from ia.gee_functions.validation import calc_area, calc_validation_score

In [ ]:
#@markdown #Run this code block!

from plotly.subplots import make_subplots

def get_lda_params(X, y, bandnames):
  sklearn_lda = LDA(n_components=1)
  LDA_clf = sklearn_lda.fit(X, y)
  LDA_fit = sklearn_lda.fit_transform(X, y)

  if np.mean(LDA_fit[:,0][y == 1]) > np.mean(LDA_fit[:,0][y == 2]):
    greater_than = False
  else:
    greater_than = True

  coefficients = list(LDA_clf.scalings_.flatten())

  pred = X.copy()

  for ind, band in enumerate(bandnames):
    pred[band] = X[band] * coefficients[ind]

  sklearn_df = pd.DataFrame(data=LDA_fit, columns=['Sklearn Prediction'])
  man_df = pd.DataFrame(data=pred.sum(1).values, columns=['Manual Prediction'])
  sklearn_df['Manual Prediction'] = man_df['Manual Prediction']
  sklearn_df['intercept'] = sklearn_df['Sklearn Prediction'] - sklearn_df['Manual Prediction']
  intercept = sklearn_df['intercept'].mode().values

  coefficients_df = pd.DataFrame({'Bandname':X.columns, 'Coefficient':coefficients})

  return intercept, coefficients_df, LDA_fit, greater_than

def get_data(data_loc, bandnames):
  df = pd.read_csv(f'{data_loc}.csv')
  df = df[bandnames + ['lc']].dropna()

  df = remove_outliers(df, bandnames)
  X = df[bandnames]            
  y = df['lc']

  return X, y

def get_histogram(X, y, classes, user_threshold=None, suggested_threshold=None, fig=None, row=None, col=None):
  
  if fig is None:
    fig = go.Figure()
  
  for cl_key in classes:
    fig.add_trace(
        go.Histogram(
            x=X[:,0][y == cl_key],
            name=classes[cl_key],
            legendgroup = cl_key
            ),
            row=row,
            col=col
            )
    
  if user_threshold is not None:
    fig.add_shape(type="line",
        x0=user_threshold, y0=0, x1=user_threshold, y1=500,
        line=dict(color="Red",width=3),
         name='Threshold selected by user',
         row=row, col=col)

  if suggested_threshold is not None:
    fig.add_shape(type="line",
        x0=suggested_threshold, y0=0, x1=suggested_threshold, y1=500,
        line=dict(color="RoyalBlue",width=3),
         name='Suggested Threshold',
         row=row, col=col)
  # Overlay both histograms
  fig.update_layout(barmode='overlay')
  # Reduce opacity to see both histograms
  fig.update_traces(opacity=0.75)
  return fig

aoi = ee.FeatureCollection(f'users/Postm087/vector/outline/outline_cdc')
aoi_coordinates = aoi.geometry().bounds().getInfo()['coordinates']
aoi_centroid = aoi.geometry().bounds().centroid(1).coordinates().getInfo()


calibration_maps = {  # Load the Land cover maps for the calibration 
    '1997':ee.Image('users/Postm087/raster/validation/cdc/C97'),
    '2000':ee.Image('users/Postm087/raster/validation/cdc/C00'),
    '2009':ee.Image('users/Postm087/raster/validation/cdc/C09'),
}

training_data_summer = {
      '1997':ee.Image('users/Postm087/ia_classification/raster/data/cdc/landsat/feature_data_lda_cdc_1997_summer'),
      '2000':ee.Image('users/Postm087/ia_classification/raster/data/cdc/landsat/feature_data_lda_cdc_2000_summer'),
      '2009':ee.Image('users/Postm087/ia_classification/raster/data/cdc/landsat/feature_data_lda_cdc_2009_summer'),
  }

training_data_winter = {
      '1997':ee.Image('users/Postm087/ia_classification/raster/data/cdc/landsat/feature_data_lda_cdc_1997_winter'),
      '2000':ee.Image('users/Postm087/ia_classification/raster/data/cdc/landsat/feature_data_lda_cdc_2000_winter'),
      '2009':ee.Image('users/Postm087/ia_classification/raster/data/cdc/landsat/feature_data_lda_cdc_2009_winter'),
  }

layer = {
    'Area of Interest':aoi.getMapId({'color':'White'})
}

bands_to_include = {
    'red':False,
    'green':False,
    'blue':False,
    'nir':False,
    'swir1':False,
    'GCVI_min':True,
    'GCVI_mean':True,
    'GCVI_max':True,
    'NDVI_min':True,
    'NDVI_mean':True,
    'NDVI_max':True,
    'NDWI_min':False,
    'NDWI_mean':False,
    'NDWI_max':False,
    'WGI_min':True,
    'WGI_mean':True,
    'WGI_max':True,
    'NDWBI_min':True,
    'NDWBI_mean':True,
    'NDWBI_max':True,
    'NDBI_min':True,
    'NDBI_mean':True,
    'NDBI_max':True,
    'NDVI_std':True,
    'GCVI_std':True,
    'NDWI_std':False,
    'WGI_std':True,
    'MTI':True,
    'slope':True,
}

bandnames = []

for key in bands_to_include:
  if bands_to_include[key]:
    bandnames += [key]

map = visualization.create_folium_map(layer, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=11, height='100%')

display(map)

In [ ]:
#@markdown #Select the year for which to perform the classification using the slider

results_folder = f'{GEE_USER_PATH}/ia_classification/raster/results/'

year = 2020 #@param {type:"slider", min:1995, max:2021, step:1}
sensor = "landsat" #@param ["landsat", "sentinel"]

if sensor == 'sentinel':
  scale=10
else:
  scale=30

summer_dates = (f'{year}-04-01', f'{year}-10-01')
winter_dates = (f'{year}-10-01', f'{year + 1}-04-01')

tasks = {}
task = create_feature_data(summer_dates, aoi, aoi_name='cdc', sensor=sensor, custom_name=f'summer_{year}')
tasks['summer'] = task
task = create_feature_data(winter_dates, aoi, aoi_name='cdc', sensor=sensor, custom_name=f'winter_{year}')
tasks['winter'] = task

while not track_task(tasks):
  pass

print('The data for classification has been generated!')

feature_data_summer = ee.Image(
        f'{GEE_USER_PATH}/ia_classification/raster/data/cdc/{sensor}/feature_data_lda_cdc_summer_{year}'
        )

feature_data_winter = ee.Image(
        f'{GEE_USER_PATH}/ia_classification/raster/data/cdc/{sensor}/feature_data_lda_cdc_winter_{year}'
        )


images = {
    f'RGB Composite of derived from satellite data for the summer of {year}':feature_data_summer.getMapId(visualization.vis_params_rgb(bands=['red', 'green', 'blue'])),
    f'RGB Composite of derived from satellite data for the winter of {year}':feature_data_winter.getMapId(visualization.vis_params_rgb(bands=['red', 'green', 'blue'])),
}

map = visualization.create_folium_map(images, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map

In [ ]:
bands_to_include = {
    'red':True,
    'green':True,
    'blue':True,
    'nir':True,
    'swir1':True,
    'GCVI_min':True,
    'GCVI_mean':True,
    'GCVI_max':True,
    'NDVI_min':True,
    'NDVI_mean':True,
    'NDVI_max':True,
    'NDWI_min':False,
    'NDWI_mean':False,
    'NDWI_max':False,
    'WGI_min':True,
    'WGI_mean':True,
    'WGI_max':True,
    'NDWBI_min':True,
    'NDWBI_mean':True,
    'NDWBI_max':True,
    'NDBI_min':True,
    'NDBI_mean':True,
    'NDBI_max':True,
    'NDVI_std':True,
    'GCVI_std':True,
    'NDWI_std':False,
    'WGI_std':True,
    'MTI':True,
    'slope':True,
}

bandnames = []

for key in bands_to_include:
  if bands_to_include[key]:
    bandnames += [key]

land_cover_classes = {
    'irrigated_trees':[7],
    'irrigated_crops':[8],
    'forest':[1, 2],
    'shrub':[3, 4],
    'rainfed_agriculture':[5, 6],
    'greenhouses':[9],
    'urban_fallow':[10],
    'water_bodies':[11],
}

tasks = {}

for cl in land_cover_classes:
  task_summer, _, _ = take_strat_sample(
      calibration_maps,
      training_data_summer,
      land_cover_classes[cl],
      aoi,
      file_name=f'sample_summer_{cl}',
      dir_name='ia_classification'
      )
  task_winter, _, _ = take_strat_sample(
      calibration_maps,
      training_data_winter,
      land_cover_classes[cl],
      aoi,
      file_name=f'sample_winter_{cl}',
      dir_name='ia_classification'
      )
  tasks[f'sample_winter_{cl}'] = task_winter
  tasks[f'sample_summer_{cl}'] = task_summer

if track_task(tasks):
  pass

if not os.getcwd() == '/content/drive/My Drive/ia_classification':
  os.chdir('/content/drive/My Drive/ia_classification')

lda_parameters_summer = {}
lda_parameters_winter = {}

for cl in land_cover_classes:
  X_summer, y_summer = get_data(f'sample_summer_{cl}', bandnames)
  X_winter, y_winter = get_data(f'sample_winter_{cl}', bandnames)

  lda_intercept_summer, lda_coefficients_summer, X_lda_sklearn_summer, gt_summer = get_lda_params(X_summer, y_summer, bandnames)
  lda_intercept_winter, lda_coefficients_winter, X_lda_sklearn_winter, gt_winter = get_lda_params(X_winter, y_winter, bandnames)

  suggested_threshold_summer = threshold_otsu(X_lda_sklearn_summer)
  suggested_threshold_winter = threshold_otsu(X_lda_sklearn_winter)

  lda_parameters_summer[f'summer_{cl}'] = [X_summer, y_summer, suggested_threshold_summer, lda_intercept_summer, lda_coefficients_summer, X_lda_sklearn_summer, gt_summer]
  lda_parameters_winter[f'winter_{cl}'] = [X_winter, y_winter, suggested_threshold_winter, lda_intercept_winter, lda_coefficients_winter, X_lda_sklearn_winter, gt_winter]
  

In [ ]:
summer_irrigated_trees_threshold = 1.4 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_irrigated_crops_threshold = 1.17 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_forest_threshold = 0.5 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_shrub_threshold = 1 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_rainfed_agriculture = 1.1 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_greenhouses = 1.5 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_urban_fallow = 0.85 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_water_bodies = 2 #@param {type:"slider", min:-3, max:3, step:0.01}

training_areas_summer = None

plot_position = {
    0:(1, 1),
    1:(1, 2),
    2:(2, 1),
    3:(2, 2),
    4:(3, 1),
    5:(3, 2),
    6:(4, 1),
    7:(4, 2),
}

fig = make_subplots(rows=4, cols=2, subplot_titles=tuple(lda_parameters_summer.keys()))

for ind, cl in enumerate(lda_parameters_summer):
  _classes = {
      1: 'other',
      2: cl,
  }

  X_summer, y_summer, suggested_threshold_summer, lda_intercept_summer, lda_coefficients_summer, X_lda_sklearn_summer, gt_summer = lda_parameters_summer[cl]
  
  if cl == 'summer_irrigated_trees':
    suggested_threshold_summer = summer_irrigated_trees_threshold
  if cl == 'summer_irrigated_crops':
    suggested_threshold_summer = summer_irrigated_crops_threshold
  if cl == 'summer_forest':
    suggested_threshold_summer = summer_forest_threshold
  if cl == 'summer_shrub':
    suggested_threshold_summer = summer_shrub_threshold
  if cl == 'summer_rainfed_agriculture':
    suggested_threshold_summer = summer_rainfed_agriculture
  if cl == 'summer_greenhouses':
    suggested_threshold_summer = summer_greenhouses
  if cl == 'summer_urban_fallow':
    suggested_threshold_summer = summer_urban_fallow
  if cl == 'summer_water_bodies':
    suggested_threshold_summer = summer_water_bodies

  ta_summer = perform_lda_scaling(feature_data_summer,lda_intercept_summer, lda_coefficients_summer, suggested_threshold_summer, gt=gt_summer)
  
  if training_areas_summer is None:
      training_areas_summer = ta_summer.select('training').rename(cl.replace('summer_', ''))
  else:
      training_areas_summer = training_areas_summer.addBands(ta_summer.select('training').rename(cl.replace('summer_', '')))

  fig = get_histogram(X_lda_sklearn_summer, y_summer, _classes, suggested_threshold_summer, fig=fig, row=plot_position[ind][0], col=plot_position[ind][1])

fig.update_layout(height=700, width=1400, showlegend=False)
fig.show()


In [ ]:
winter_irrigated_trees_threshold = 1.5  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_irrigated_crops_threshold = 1.25  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_forest_threshold = 0.74  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_shrub_threshold = 1  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_rainfed_agriculture = 1.5  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_greenhouses = 1.5  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_urban_fallow = 0.85  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_water_bodies = 1.5  # @param {type:"slider", min:-3, max:3, step:0.01}

training_areas_winter = None

plot_position = {
    0: (1, 1),
    1: (1, 2),
    2: (2, 1),
    3: (2, 2),
    4: (3, 1),
    5: (3, 2),
    6: (4, 1),
    7: (4, 2),
}

fig = make_subplots(rows=4, cols=2, subplot_titles=tuple(lda_parameters_winter.keys()))

for ind, cl in enumerate(lda_parameters_winter):

    X_winter, y_winter, suggested_threshold_winter, lda_intercept_winter, lda_coefficients_winter, X_lda_sklearn_winter, gt_winter = \
    lda_parameters_winter[cl]

    if gt_winter:
      _classes = {
          1: 'other',
          2: cl,
      }
    else:
      _classes = {
          1: cl,
          2: 'other',
      }

    if cl == 'winter_irrigated_trees':
        suggested_threshold_winter = winter_irrigated_trees_threshold
    if cl == 'winter_irrigated_crops':
        suggested_threshold_winter = winter_irrigated_crops_threshold
    if cl == 'winter_forest':
        suggested_threshold_winter = winter_forest_threshold
    if cl == 'winter_shrub':
        suggested_threshold_winter = winter_shrub_threshold
    if cl == 'winter_rainfed_agriculture':
        suggested_threshold_winter = winter_rainfed_agriculture
    if cl == 'winter_greenhouses':
        suggested_threshold_winter = winter_greenhouses
    if cl == 'winter_urban_fallow':
        suggested_threshold_winter = winter_urban_fallow
    if cl == 'winter_water_bodies':
        suggested_threshold_winter = winter_water_bodies

    ta_winter = perform_lda_scaling(feature_data_winter, lda_intercept_winter, lda_coefficients_winter,
                                    suggested_threshold_winter, gt=gt_winter)
    if training_areas_winter is None:
      training_areas_winter = ta_winter.select('training').rename(cl.replace('winter_', ''))
    else:
      training_areas_winter = training_areas_winter.addBands(ta_winter.select('training').rename(cl.replace('winter_', '')))

    fig = get_histogram(X_lda_sklearn_winter, y_winter, _classes, suggested_threshold_winter, fig=fig,
                        row=plot_position[ind][0], col=plot_position[ind][1])

fig.update_layout(height=700, width=1400, showlegend=False)
fig.show()


In [ ]:
lc_classes = {
    'irrigated_trees':1,
    'irrigated_crops':2,
    'forest':3,
    'shrub':4,
    'rainfed_agriculture':5,
    'greenhouses':6,
    'urban_fallow':7,
    'water_bodies':8,
}

def combine_training_areas(img_training_areas, classes):
  # Combine the training areas in to single image 

  img_training_areas_summed = img_training_areas.reduce('sum')
  img_training_areas_combined = ee.Image(0)

  for ind, cl in enumerate(classes):
    img_training_areas_combined = img_training_areas_combined.where(img_training_areas.select(cl).eq(1).And(img_training_areas_summed.eq(1)), lc_classes[cl])

  return img_training_areas_combined

task_summer = export_to_asset(
            combine_training_areas(training_areas_summer, lc_classes),
            'image',
            f"results/random_forest/cdc_lda/training/training_summer_{year}",
            aoi_coordinates,
            scale,
            overwrite=True
        )

task_winter = export_to_asset(
            combine_training_areas(training_areas_winter, lc_classes),
            'image',
            f"results/random_forest/cdc_lda/training/training_winter_{year}",
            aoi_coordinates,
            scale,
            overwrite=True
        )

tasks = {
    'training areas summer':task_summer,
    'training areas winter':task_winter,
}

track_task(tasks)

In [ ]:
lc_classes = {
        0: 'not selected',
        1: 'irrigated_trees',
        2: 'irrigated_crops',
        3: 'forest',
        4: 'shrub',
        5: 'rainfed_agriculture',
        6: 'greenhouses',
        7: 'urban_fallow',
        8: 'water_bodies'
    }


palette = ['#FFFFFF', '64C3FF', '6464FE', '009600', '824B32', 'F5D7A5', 'FFFF00','AA0F6E', '00008b']

summer_training = ee.Image(f'{results_folder}random_forest/cdc_lda/training/training_summer_{year}')
winter_training = ee.Image(f'{results_folder}random_forest/cdc_lda/training/training_winter_{year}')

images = {
    f'RGB Composite of derived from satellite data for the summer of {year}':feature_data_summer.getMapId(visualization.vis_params_rgb(bands=['red', 'green', 'blue'])),
    f'RGB Composite of derived from satellite data for the winter of {year}':feature_data_winter.getMapId(visualization.vis_params_rgb(bands=['red', 'green', 'blue'])),
    f'Training Areas Summer':summer_training.clip(aoi).getMapId(visualization.vis_params_cp('constant', 0, 8, palette)),
    f'Training Areas Winter':winter_training.clip(aoi).getMapId(visualization.vis_params_cp('constant', 0, 8, palette)),      
}

map = visualization.create_folium_map(images, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map = visualization.create_categorical_legend(map,palette, lc_classes)
map 

In [ ]:
#@markdown ---
#@title ### Classification Parameters
#@markdown ### Number of Trees
no_trees = 150 #@param {type:"slider", min:0, max:500, step:10}
#@markdown ### Variables per Split
vps = 6 #@param {type:"slider", min:2, max:10, step:1}
#@markdown ### Bagging Fraction
bf = 0.2 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown ### Minimum Number of Training Points
min_tp = 500 #@param {type:"slider", min:100, max:5000, step:100}
#@markdown ### Maximum Number of Training Points
max_tp = 2000 #@param {type:"slider", min:100, max:75000, step:100}
classification_tasks = {}

classification_data = {
    'summer':{
        'feature_data':feature_data_summer,
        'training_data':summer_training.rename('training').reproject(feature_data_summer.projection()),
        },
    'winter':{
        'feature_data':feature_data_winter,
        'training_data':winter_training.rename('training').reproject(feature_data_winter.projection()),
    }
}

classification_tasks = {}
classifiers = {}

for season in classification_data:
  training = classification_data[season]['training_data']
  training = training.addBands(training)

  classification_task, clf = classify_irrigated_areas(
        classification_data[season]['feature_data'],
        training,
        aoi,
        aoi_name='cdc_lda',
        season=season,
        year=str(year),
        no_trees=no_trees,
        bag_fraction=bf,
        vps=vps,
        min_tp=min_tp,
        max_tp=max_tp,
        overwrite=True
        )
  classifiers[season] = clf
  classification_tasks[season] = classification_task

track_task(classification_tasks)

lc_classes_headers = [
                      '',
        'irrigated_trees',
        'irrigated_crops',
        'forest',
        'shrub',
        'rainfed_agriculture',
        'greenhouses',
        'urban_fallow',
        'water_bodies'
    ]

lc_classes_index = [
        'irrigated_trees',
        'irrigated_crops',
        'forest',
        'shrub',
        'rainfed_agriculture',
        'greenhouses',
        'urban_fallow',
        'water_bodies'
    ]


for clf in classifiers:
  conf_matrix = classifiers[clf].confusionMatrix()
  accuracy = conf_matrix.accuracy().getInfo()
  matrix = conf_matrix.getInfo()
  df = pd.DataFrame(matrix).drop(0, 1).drop(0)
  df['Index'] = lc_classes_index
  df = df.set_index('Index').reset_index()

  fig = go.Figure(layout={'title':f'Confusion Matrix {clf} Classifier. Accuracy: {round(accuracy * 100)}%'},
      data=[go.Table(
    header=dict(values=lc_classes_headers,
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=df.transpose().values.tolist(),
               fill_color='lavender',
               align='left'))
  ])

  fig['layout'].update(margin=dict(b=0), height=320)

  fig.show()

In [ ]:
lc_classes = {
        0: 'NA',
        1: 'irrigated_trees',
        2: 'irrigated_crops',
        3: 'forest',
        4: 'shrub',
        5: 'rainfed_agriculture',
        6: 'greenhouses',
        7: 'urban_fallow',
        8: 'water_bodies'
    }


palette = ['FFFFFF','64C3FF', '6464FE', '009600', '824B32', 'F5D7A5', 'FFFF00','AA0F6E', '00008b']


summer_clf = ee.Image(f'{results_folder}random_forest/cdc_lda/ia_random_forest_{no_trees}tr_{vps}vps_{int(bf*100)}bf_cdc_lda_summer_{year}')
winter_clf = ee.Image(f'{results_folder}random_forest/cdc_lda/ia_random_forest_{no_trees}tr_{vps}vps_{int(bf*100)}bf_cdc_lda_winter_{year}')

images_clf  = {
    f'RGB Composite of derived from satellite data for the summer of {year}':feature_data_summer.getMapId(visualization.vis_params_rgb(bands=['red', 'green', 'blue'])),
    f'RGB Composite of derived from satellite data for the winter of {year}':feature_data_winter.getMapId(visualization.vis_params_rgb(bands=['red', 'green', 'blue'])),
    'Summer RF Classification results': summer_clf.getMapId(visualization.vis_params_cp('rf_all_classes', 0, 8, palette)),
    'Winter RF Classification results': winter_clf.getMapId(visualization.vis_params_cp('rf_all_classes', 0, 8, palette)),
}

map = visualization.create_folium_map(images_clf, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map = visualization.create_categorical_legend(map, palette, lc_classes)
map 

In [ ]:
#@markdown # Combine Irrigated Area Maps

ia_summer = summer_clf.select('irrigated_area')
ia_winter = winter_clf.select('irrigated_area')
task = join_seasonal_irrigated_areas(
    ia_summer,
    ia_winter,
    f'cdc_lda',
    year,
    aoi,
    overwrite=True,
    export_method='asset',
  )
track_task(task)

ia_classes = {
        0: 'Not Irrigated',
        1: 'Year Round Irrigated Crops',
        2: 'Year Round Irrigated Trees',
        3: 'Summer Irrigated Crops',
        4: 'Summer Irrigated Trees',
        5: 'Winter Irrigated Crops',
        6: 'Winter Irrigated Trees',
        7: 'Uncertain Areas',
    }

palette = [
           '000000',
           '211cff',
           '20b407',
           '0aaeff',
           '5bff0a',
           '64d3ff',
           '86ffa7',
           'ff7e9b'
           ]

ia_year = ee.Image(f'{results_folder}irrigated_area/{sensor}/cdc_lda/irrigated_areas_{sensor}/cdc_lda_{year}').clip(aoi)

images_results = {
      'Irrigated Areas Overview': ia_year.getMapId(visualization.vis_params_cp('ia_year', 0, 7, palette)),
  }

total_irrigated_area =  calc_area(ia_year.select('ia_year').gt(0), aoi).getInfo()

for cl in ia_classes:
  class_area =  calc_area(ia_year.select('ia_year').eq(cl), aoi).getInfo()

  ia_classes[cl] = f"{ia_classes[cl]} ({round(class_area)} ha.)"


map = visualization.create_folium_map(images_results, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map = visualization.create_categorical_legend(map, palette, ia_classes)
map = visualization.create_hectares_label(map, round(total_irrigated_area))
map

In [ ]:
#@markdown # Export irrigated area map to Google drive
#@markdown ---   
file_name = "irrigated_area_map" #@param {type:"string"}
folder_name = "ia_detection_result" #@param {type:"string"}

task = ee.batch.Export.image.toDrive(
    ia_year,
    description=file_name,
    folder=folder_name,
    maxPixels=1000000000000,
    region=aoi_coordinates
    ).start()